In [1]:
from llama_index.finetuning import EmbeddingQAFinetuneDataset
from datasets import load_from_disk

In [2]:
dataset = load_from_disk('../data/processed/dataset')

In [3]:
dataset['train']

Dataset({
    features: ['question', 'ground_truth', 'page', 'document'],
    num_rows: 1423
})

In [4]:
import chromadb
client = chromadb.PersistentClient(f'../data/db')
collection = client.get_collection('texts_e5')
docs = collection.get()

In [5]:
def get_doc_ids(example):
    docs = collection.get(where={
        '$and': [
            {'page': example['page']},
            {'document_name': example['document']}
            ]
            })
    return docs['ids']

In [6]:
def filter(queries, relevant_docs):
    t_query = list(relevant_docs.keys()).copy()
    t_docs = list(relevant_docs.values()).copy()
    for query, docs in zip(t_query, t_docs):
        if len(docs) == 0:
            queries.pop(query)
            relevant_docs.pop(query)

In [7]:
corpus = {k: doc_name['document_name'] + '\n' + v for k, v, doc_name in zip(docs['ids'], docs['documents'], docs['metadatas'])}
queries_train = {str(i):dataset['train'][i]['question'] for i in range(len(dataset['train']))}
relevant_docs_train = {
    str(i):get_doc_ids(dataset['train'][i]) for i in range(len(dataset['train']))
}
filter(queries_train, relevant_docs_train)

In [8]:
queries_val = {str(i):dataset['val'][i]['question'] for i in range(len(dataset['val']))}
relevant_docs_val = {
    str(i):get_doc_ids(dataset['val'][i]) for i in range(len(dataset['val']))
}
filter(queries_val, relevant_docs_val)

In [9]:
queries_test = {str(i):dataset['test'][i]['question'] for i in range(len(dataset['test']))}
relevant_docs_test = {
    str(i):get_doc_ids(dataset['test'][i]) for i in range(len(dataset['test']))
}
filter(queries_test, relevant_docs_test)

In [10]:
dataset_train = EmbeddingQAFinetuneDataset(queries=queries_train, corpus=corpus, relevant_docs=relevant_docs_train)
dataset_val = EmbeddingQAFinetuneDataset(queries=queries_val, corpus=corpus, relevant_docs=relevant_docs_val)
dataset_test = EmbeddingQAFinetuneDataset(queries=queries_test, corpus=corpus, relevant_docs=relevant_docs_test)

In [27]:
dataset_train.save_json('train_test.json')
dataset_val.save_json('val_test.json')
dataset_test.save_json('test_test.json')

In [28]:
import tempfile
import json

In [29]:
with tempfile.NamedTemporaryFile(mode='w', delete=False) as tmp_file:
    # dataset_stage.save_json(tmp_file.name)
    json.dump(dataset_val.dict(), tmp_file, ensure_ascii=False, indent=4)
    # s3_client.upload_file(tmp_file.name, params.bucket_name, f'{params.output_data_s3}/{stage}.json')

In [30]:
tmp_file.name

'/var/folders/r_/bq4swdms3vj5wr80yf1vm4nh0000gn/T/tmpqf_wkk6f'

In [14]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer

def evaluate_st(
    dataset,
    model_id,
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs
    evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)
    model = SentenceTransformer(model_id)
    return evaluator(model)

In [ ]:
SentenceTransformer.h

In [ ]:
from sentence_transformers import SentenceTransformer

In [15]:
train_metrics = evaluate_st(dataset_train, "intfloat/multilingual-e5-large")

KeyboardInterrupt: 

In [ ]:
val_metrics = evaluate_st(dataset_val, "intfloat/multilingual-e5-large")

In [11]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

In [12]:
finetune_engine = SentenceTransformersFinetuneEngine(
    dataset_train, # Dataset to be trained on
    model_id="intfloat/multilingual-e5-large", # HuggingFace reference to base embeddings model
    model_output_path="llama_model_v1", # Output directory for fine-tuned embeddings model
    val_dataset=dataset_val, # Dataset to validate on
    epochs=4,
    batch_size=2,
)

In [22]:
import boto3
import json

In [24]:
file_content

'{\n    "queries": {\n        "0": "query: Was könnte ein Zweck dieses Diagramms sein?",\n        "1": "query: Mit welchem Mantelmaterial werden LEONI Adascart® Sensor ungeschirmt oder geschirmt Kabel ausgestattet, um eine Betriebstemperatur von +150 °C zu unterstützen?",\n        "2": "query: Was repräsentiert die Beschriftung \'a\' in der Legende unter dem Bild?",\n        "3": "query: Was ist der nächste Schritt, nachdem das Werkzeug in der Presse installiert wurde?",\n        "4": "query: Welche Norm regelt die Spezifikationen von FLY-Leitungen mit PVC-Isolierung?",\n        "5": "query: Welche maximale Anzahl an Einzeldrähten weist eine der aufgelisteten Leitungen auf?",\n        "6": "query: Welches Adern- und Mantelmaterial wird für Kabel mit einem Aderdurchmesser von 2,65 mm verwendet?",\n        "7": "query: Was ist der Zweck der Messstrecken in Abbildung PC-02?",\n        "8": "query: Wie ist der Abstand der Crimpflankenenden (CFE) definiert?",\n        "9": "query: Was verst

In [ ]:
tmp_file.name

In [37]:
s3 = boto3.resource('s3')
obj = s3.Object('tcr-internal', 'dmitrii/data/train.json')
file_content = obj.get()['Body'].read().decode('utf-8')
a = json.loads(file_content)

In [38]:
file_content

'{\n    "queries": {\n        "0": "query: Was k\\u00f6nnte ein Zweck dieses Diagramms sein?",\n        "1": "query: Mit welchem Mantelmaterial werden LEONI Adascart\\u00ae Sensor ungeschirmt oder geschirmt Kabel ausgestattet, um eine Betriebstemperatur von +150 \\u00b0C zu unterst\\u00fctzen?",\n        "2": "query: Was repr\\u00e4sentiert die Beschriftung \'a\' in der Legende unter dem Bild?",\n        "3": "query: Was ist der n\\u00e4chste Schritt, nachdem das Werkzeug in der Presse installiert wurde?",\n        "4": "query: Welche Norm regelt die Spezifikationen von FLY-Leitungen mit PVC-Isolierung?",\n        "5": "query: Welche maximale Anzahl an Einzeldr\\u00e4hten weist eine der aufgelisteten Leitungen auf?",\n        "6": "query: Welches Adern- und Mantelmaterial wird f\\u00fcr Kabel mit einem Aderdurchmesser von 2,65 mm verwendet?",\n        "7": "query: Was ist der Zweck der Messstrecken in Abbildung PC-02?",\n        "8": "query: Wie ist der Abstand der Crimpflankenenden (C

In [39]:
a

{'queries': {'0': 'query: Was könnte ein Zweck dieses Diagramms sein?',
  '1': 'query: Mit welchem Mantelmaterial werden LEONI Adascart® Sensor ungeschirmt oder geschirmt Kabel ausgestattet, um eine Betriebstemperatur von +150 °C zu unterstützen?',
  '2': "query: Was repräsentiert die Beschriftung 'a' in der Legende unter dem Bild?",
  '3': 'query: Was ist der nächste Schritt, nachdem das Werkzeug in der Presse installiert wurde?',
  '4': 'query: Welche Norm regelt die Spezifikationen von FLY-Leitungen mit PVC-Isolierung?',
  '5': 'query: Welche maximale Anzahl an Einzeldrähten weist eine der aufgelisteten Leitungen auf?',
  '6': 'query: Welches Adern- und Mantelmaterial wird für Kabel mit einem Aderdurchmesser von 2,65 mm verwendet?',
  '7': 'query: Was ist der Zweck der Messstrecken in Abbildung PC-02?',
  '8': 'query: Wie ist der Abstand der Crimpflankenenden (CFE) definiert?',
  '9': 'query: Was versteht man unter der messbaren Crimpbreite (Cbm)?',
  '10': 'query: Was ermöglicht di

In [19]:
finetune_engine.examples[0].texts

['Was könnte ein Zweck dieses Diagramms sein?',
 'GS 95006-7-5 Kfz-Kontaktierungen Überwachungskriterien von Crimp-1\nGS 95006-7-5:2017-04      LV 214-4: 2017-04\n7\nLegende:\na  Bereich, in dem F max  liegt, zur Ermittlung der relativen Streuung der Spitzenkraft\nb  Messwerte der Crimp-Kraftkurve  für Gut-Crimps\ns  Weg\nF  Kraft\nBild 3 Crimp-Kraftverlauf von Gut-Crimps\nUT = X max\nF\nS\nb\na F max']

In [15]:
for i in finetune_engine.loader:
    print(i)
    break

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'sentence_transformers.readers.InputExample.InputExample'>

In [13]:
import torch
finetune_engine.model._target_device = torch.device('mps')
finetune_engine.model._target_device

device(type='mps')

In [14]:
finetune_engine.finetune()

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/700 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 9.74 GB, other allocations: 25.79 GB, max allowed: 36.27 GB). Tried to allocate 976.57 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [31]:
import os

In [33]:
os.remove(tmp_file.name)

In [44]:
def download_s3_folder(bucket_name, s3_folder, local_dir):
    """
    Download an entire folder from an S3 bucket to a local directory.

    :param bucket_name: Name of the S3 bucket.
    :param s3_folder: Folder path in the S3 bucket.
    :param local_dir: Local directory to which the folder will be downloaded.
    """
    s3_client = boto3.client('s3')
    paginator = s3_client.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=bucket_name, Prefix=s3_folder):
        for obj in page.get('Contents', []):
            local_file_path = os.path.join(local_dir, os.path.relpath(obj['Key'], s3_folder))            
            local_file_dir = os.path.dirname(local_file_path)
            if not os.path.exists(local_file_dir):
                os.makedirs(local_file_dir)
            
            s3_client.download_file(bucket_name, obj['Key'], local_file_path)
            print(f"Downloaded {obj['Key']} to {local_file_path}")


bucket_name = 'tcr-internal'
s3_folder = 'dmitrii/models/retriever-e5-finetuned/'
local_dir = 'models/retriever-e5-finetuned/'

download_s3_folder(bucket_name, s3_folder, local_dir)

Downloaded dmitrii/models/retriever-e5-finetuned/1_Pooling/config.json to models/retriever-e5-finetuned/1_Pooling/config.json
Downloaded dmitrii/models/retriever-e5-finetuned/README.md to models/retriever-e5-finetuned/README.md
Downloaded dmitrii/models/retriever-e5-finetuned/config.json to models/retriever-e5-finetuned/config.json
Downloaded dmitrii/models/retriever-e5-finetuned/config_sentence_transformers.json to models/retriever-e5-finetuned/config_sentence_transformers.json
Downloaded dmitrii/models/retriever-e5-finetuned/eval/Information-Retrieval_evaluation_results.csv to models/retriever-e5-finetuned/eval/Information-Retrieval_evaluation_results.csv
Downloaded dmitrii/models/retriever-e5-finetuned/model.safetensors to models/retriever-e5-finetuned/model.safetensors
Downloaded dmitrii/models/retriever-e5-finetuned/modules.json to models/retriever-e5-finetuned/modules.json
Downloaded dmitrii/models/retriever-e5-finetuned/sentence_bert_config.json to models/retriever-e5-finetuned/

In [41]:
from aim import Run

run = Run(experiment='train_retriever', capture_terminal_logs=False)

In [42]:
import pandas as pd

In [45]:
df = pd.read_csv('./models/retriever-e5-finetuned/eval/Information-Retrieval_evaluation_results.csv')

In [46]:
df.columns

Index(['epoch', 'steps', 'cos_sim-Accuracy@1', 'cos_sim-Accuracy@3',
       'cos_sim-Precision@1', 'cos_sim-Recall@1', 'cos_sim-Precision@3',
       'cos_sim-Recall@3', 'cos_sim-MRR@1', 'cos_sim-MRR@3', 'cos_sim-NDCG@1',
       'cos_sim-NDCG@3', 'cos_sim-MAP@1', 'cos_sim-MAP@3', 'cos_sim-MAP@100',
       'dot_score-Accuracy@1', 'dot_score-Accuracy@3', 'dot_score-Precision@1',
       'dot_score-Recall@1', 'dot_score-Precision@3', 'dot_score-Recall@3',
       'dot_score-MRR@1', 'dot_score-MRR@3', 'dot_score-NDCG@1',
       'dot_score-NDCG@3', 'dot_score-MAP@1', 'dot_score-MAP@3',
       'dot_score-MAP@100'],
      dtype='object')

In [55]:
for _, row in df.filter(like='cos_sim-').iterrows():
    for k, v in row.items():
        run.track(v, name=k.replace('cos_sim-', ''))

In [54]:
for i in row.items():
    print(i)

('cos_sim-Accuracy@1', 0.6078571428571429)
('cos_sim-Accuracy@3', 0.8114285714285714)
('cos_sim-Precision@1', 0.6078571428571429)
('cos_sim-Recall@1', 0.4831547619047618)
('cos_sim-Precision@3', 0.3092857142857143)
('cos_sim-Recall@3', 0.7051190476190476)
('cos_sim-MRR@1', 0.6078571428571429)
('cos_sim-MRR@3', 0.7005952380952389)
('cos_sim-NDCG@1', 0.6078571428571429)
('cos_sim-NDCG@3', 0.6573346553026075)
('cos_sim-MAP@1', 0.6078571428571429)
('cos_sim-MAP@3', 0.6103174603174603)
('cos_sim-MAP@100', 0.6548095835205994)
